In [2]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import pandas as pd
from lxml import html
import numpy as np
import time

In [5]:
ticker_selection = pd.read_csv('tickers_only.csv')
tickers = ticker_selection['ticker']
tickers

0          A
1         AA
2        AAC
3       AACG
4       AADI
        ... 
5873    ZWRK
5874      ZY
5875    ZYME
5876    ZYNE
5877    ZYXI
Name: ticker, Length: 5878, dtype: object

In [20]:
driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
driver.get('https://stockanalysis.com/stocks/')

In [23]:
players = driver.find_elements_by_xpath('//*[@id="main"]/div/div/div[2]/table/tbody/tr[1]/td[1]/a')
href = players.get_attribute('href')
print(href)


AttributeError: 'list' object has no attribute 'get_attribute'

In [12]:
players = driver.find_elements_by_xpath('//*[@id="main"]/div/div/div[2]/table/tbody/tr/td[1]/a')

players_list = []
for p in range(len(players)):
    players_list.append(players[p].text)

players_list

['A',
 'AA',
 'AAC',
 'AACG',
 'AAIC',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAQC',
 'AAT',
 'AATC',
 'AAU',
 'AAWW',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABEV',
 'ABG',
 'ABGI',
 'ABIO',
 'ABM',
 'ABMD',
 'ABNB',
 'ABOS',
 'ABR',
 'ABSI',
 'ABST',
 'ABT',
 'ABTX',
 'ABUS',
 'ABVC',
 'AC',
 'ACA',
 'ACAD',
 'ACAH',
 'ACB',
 'ACBA',
 'ACBI',
 'ACC',
 'ACCD',
 'ACCO',
 'ACEL',
 'ACER',
 'ACET',
 'ACEV',
 'ACGL',
 'ACH',
 'ACHC',
 'ACHL',
 'ACHV',
 'ACI',
 'ACIC',
 'ACII',
 'ACIU',
 'ACIW',
 'ACKIT',
 'ACLS',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACOR',
 'ACQR',
 'ACR',
 'ACRE',
 'ACRS',
 'ACRX',
 'ACST',
 'ACTD',
 'ACTG',
 'ACU',
 'ACVA',
 'ACXP',
 'ACY',
 'ADAG',
 'ADAP',
 'ADBE',
 'ADC',
 'ADCT',
 'ADER',
 'ADES',
 'ADEX',
 'ADF',
 'ADGI',
 'ADI',
 'ADIL',
 'ADM',
 'ADMA',
 'ADMP',
 'ADMS',
 'ADN',
 'ADNT',
 'ADOC',
 'ADP',
 'ADPT',
 'ADRA',
 'ADS',
 'ADSK',
 'ADT',
 'ADTN',
 'ADTX',
 'ADUS',
 'ADV',
 'ADVM',
 'ADXN',
 'AD

In [ ]:
//*[@id="main"]/div/div/nav/div/select

In [34]:
web = "https://stockanalysis.com/stocks/"
driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
driver.get(web)

sel = Select(driver.find_element_by_xpath('//select[@name="perpage"]'))
sel.select_by_visible_text("10000")

In [36]:
import time

web = "https://stockanalysis.com/stocks/"
driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
driver.get(web)
sel = Select(driver.find_element_by_xpath('//select[@name="perpage"]'))
sel.select_by_visible_text("10000")

time.sleep(5)

ticker_list = []
company_name_list = []
industry_list = []

## starting to find elements
ticker = driver.find_elements_by_xpath('//*[@id="main"]/div/div/div[2]/table/tbody/tr/td[1]/a')
for a in range(len(ticker)):
    ticker_list.append(ticker[a].text)

company_name = driver.find_elements_by_xpath('//*[@id="main"]/div/div/div[2]/table/tbody/tr/td[2]')
for b in range(len(company_name)):
    company_name_list.append(company_name[b].text)

industry = driver.find_elements_by_xpath('//*[@id="main"]/div/div/div[2]/table/tbody/tr/td[3]')
for c in range(len(industry)):
    industry_list.append(industry[c].text)

## Creating dataframes so I can join this all together
ticker_df = pd.DataFrame(ticker_list)
company_name_df = pd.DataFrame(company_name_list)
industry_df = pd.DataFrame(industry_list)
big_df = pd.concat([ticker_df, company_name_df, industry_df], axis=1)

In [43]:
big_df['url'] = "https://stockanalysis.com/stocks/" + big_df['ticker'] + "/"
big_df

,ticker,company_name,industry,full,url
0,A,Agilent Technologies,Diagnostics & Research,A - Agilent Technologies,https://stockanalysis.com/stocks/A/
1,AA,Alcoa,Aluminum,AA - Alcoa,https://stockanalysis.com/stocks/AA/
2,AAC,Ares Acquisition,Shell Companies,AAC - Ares Acquisition,https://stockanalysis.com/stocks/AAC/
3,AACG,ATA Creativity Global,Education & Training Services,AACG - ATA Creativity Global,https://stockanalysis.com/stocks/AACG/
4,AAIC,Arlington Asset Investment,REIT-Mortgage,AAIC - Arlington Asset Investment,https://stockanalysis.com/stocks/AAIC/
...,...,...,...,...,...
5854,ZWRK,Z-Work Acquisition,Shell Companies,ZWRK - Z-Work Acquisition,https://stockanalysis.com/stocks/ZWRK/
5855,ZY,Zymergen,Chemicals,ZY - Zymergen,https://stockanalysis.com/stocks/ZY/
5856,ZYME,Zymeworks,Biotechnology,ZYME - Zymeworks,https://stockanalysis.com/stocks/ZYME/
5857,ZYNE,Zynerba Pharmaceuticals,Drug Manufacturers-Specialty & Generic,ZYNE - Zynerba Pharmaceuticals,https://stockanalysis.com/stocks/ZYNE/


In [40]:
big_df.columns = ['ticker', 'company_name', 'industry']
big_df['full'] = big_df['ticker'] + " - " + big_df['company_name']
big_df['url'] = "https://stockanalysis.com/stocks/" + big['ticker'] + "/"
big_df

ValueError: Length mismatch: Expected axis has 4 elements, new values have 3 elements

In [ ]:
salaries = driver.find_elements_by_xpath('//td[@class="hh-salaries-sorted"]')

salaries_list = []
for s in range(len(salaries)):
    salaries_list.append(salaries[s].text)

## Example 2

In [90]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

In [91]:
driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
#driver = webdriver.Firefox(r'C:\Users\rober\Anaconda3\bin\geckodriver')
driver.get('http://www.platinum.matthey.com/prices/price-charts')

In [42]:
# Wait 20 seconds for page to load
timeout = 30
try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, "//*[@id='chart-content']/div/div[1]/div/div/table/tbody/tr[1]/td[1]")))
except TimeoutException:
    print('Timed out waiting for page to load')
    driver.quit()

Timed out waiting for page to load


In [109]:
# find_elements_by_xpath returns an array of selenium objects.
dates_element = driver.find_elements_by_xpath("""//*[@id="chart-content"]/div/div[1]/div/div/table/tbody/tr""")
#dates_string = dates_element.text
dates_list = []
for p in range(len(dates_element)):
    dates_list.append(dates_element[p].text)

In [110]:
dates_list

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

## lets try the table

In [111]:
driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
#driver = webdriver.Firefox(r'C:\Users\rober\Anaconda3\bin\geckodriver')
driver.get('http://www.platinum.matthey.com/prices/price-tables')

In [113]:
# find_elements_by_xpath returns an array of selenium objects.
dates_element = driver.find_elements_by_xpath("""//*[@id="priceTablesFrame"]/table[1]/tbody/tr[8]/th[2]""")
#dates_string = dates_element.text
dates_list = []
for p in range(len(dates_element)):
    dates_list.append(dates_element[p].text)

In [114]:
dates_list

['12nd']

## Metal Prices

In [34]:
option = webdriver.ChromeOptions()
option.add_argument(“ — incognito”)
driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
#driver = webdriver.Firefox(r'C:\Users\rober\Anaconda3\bin\geckodriver')
driver.get('http://www.platinum.matthey.com/prices/price-charts')

SyntaxError: invalid character in identifier (<ipython-input-34-0dc4660eb39a>, line 2)

In [32]:
dates = driver.find_elements_by_xpath('//*[@id="chart-content"]/div/div[1]/div/svg/g[2]/g/g/text')

In [33]:
dates = []
for p in range(len(dates)):
    dates_list.append(dates[p].text)

dates

[]

## xpath

In [138]:
request = requests.get('http://www.platinum.matthey.com/prices/price-tables')
data = html.fromstring(request.text)

In [77]:
dates = data.xpath("//*[@id='chart-content']/div/div[2]/text")
print(dates)

[]


In [151]:
request = requests.get('http://www.platinum.matthey.com/prices/price-tables')
data = html.fromstring(request.text)

# Xpath
alldata =[]

for i in data.xpath("//tr[@class]"):
    day = i.xpath('th[2]/text()')
    url = i.xpath('td[4]/text()')
    #year = i.xpath('span[2]/text()')   
    #print(title, url, year)
    alldata.append([day, url])
    
len(alldata)

80

In [209]:
metal_df = pd.DataFrame(alldata)
metal_df.columns = ['day', 'price - USD per troy ounce']
metal_df = metal_df.tail(-2)
metal_df = metal_df.explode('price - USD per troy ounce')
metal_df = metal_df.explode('day')
metal_df['day'] = metal_df['day'].str.replace('\r\n ', '')
metal_df['price - USD per troy ounce'] = metal_df['price - USD per troy ounce'].str.replace('\r\n ', '')
metal_df['day'] = metal_df['day'].str.strip()
metal_df['len'] = metal_df['day'].str.len()
metal_df['count_blank'] = np.where((metal_df['day'].isnull()) & (metal_df['price - USD per troy ounce'].isnull()), 1, 0)
metal_df = metal_df[metal_df.len != 0]
metal_df['flag_delete_row_1'] = np.where(metal_df['len'] > 2, 1, 0)
metal_df['flag_delete_row_2'] = np.where((metal_df['len'].isnull() == True) & (metal_df['count_blank'] == 0), 1, 0)
metal_df = metal_df[metal_df.flag_delete_row_1 != 1]
metal_df = metal_df[metal_df.flag_delete_row_2 != 1]
metal_df['blank_cum_sum'] = metal_df['count_blank'].cumsum()

##dictionary of values I want to map
metal_dict = {0: 'Platinum', 1: 'Palladium', 2: 'Rhodium', 3: 'Iridium', 4: 'Ruthenium'}
metal_df['metal_name'] = metal_df['blank_cum_sum'].map(metal_dict)
metal_df = metal_df[metal_df.count_blank != 1]
metal_df.head(30)

,day,price - USD per troy ounce,len,count_blank,flag_delete_row_1,flag_delete_row_2,blank_cum_sum,metal_name
2,1,1094,1.0,0,0,0,0,Platinum
3,2,1094,1.0,0,0,0,0,Platinum
4,5,1104,1.0,0,0,0,0,Platinum
5,6,1117,1.0,0,0,0,0,Platinum
6,7,1094,1.0,0,0,0,0,Platinum
7,8,1080,1.0,0,0,0,0,Platinum
8,9,1095,1.0,0,0,0,0,Platinum
9,12,1113,2.0,0,0,0,0,Platinum
10,13,1116,2.0,0,0,0,0,Platinum
11,14,1135,2.0,0,0,0,0,Platinum


In [157]:
print(metal_df['date'][0])

['London']


In [122]:
for i in alldata:
    url = (i[0].replace('\r\n        ', ''))
    print(url)

AttributeError: 'list' object has no attribute 'replace'